In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

import seaborn as sns
import matplotlib.pyplot as plt

import module
import importlib
try:
    importlib.reload(module) # reload module
except NameError:
    pass

# Specify Input

In [3]:
date= '2023-05-08'# to get parquet

end_date= '2023-05-01' # last month at which you want to get EOC
end_date= pd.to_datetime(end_date) + pd.offsets.MonthEnd(0) # get last day of end date
start_date= end_date - pd.offsets.MonthBegin(13) # get prev 13 month

print(f'date = {start_date.date()} - {end_date.date()}')

date = 2022-05-01 - 2023-05-31


# Load DF

In [4]:
path= Path(f'output/member_data_{date}.parquet')
df= pd.read_parquet(path)

df.sample(3)

,active_apr_2023,active_dec_2022,active_feb_2023,active_jan_2023,active_mar_2023,active_nov_2022,active_oct_2022,addon_1,addon_2,age,...,income,index,is_cpt,is_next_contract,job,membership_code,membership_status,phone,product,start_date
333,True,True,True,True,True,False,False,None,None,25.0,...,NaN,10.0,NaN,False,None,DLX,Paid Member,082133897094,[DLX.20] 20 Months Membership Deluxe - 2022 - NST,2022-12-31
7754,False,False,False,False,False,False,False,None,None,27.0,...,NaN,NaN,NaN,False,employee,CLB DLX PRO BDL,Paid Member,None,[CLB.DLX.PRO.BDL.20] Ramadhan Bundle - 20 Mont...,2020-07-09
5466,False,True,False,False,False,True,True,Proskill,None,21.0,...,NaN,38.0,NaN,True,college student,GO DLX PRO REP,Paid Member,08111654444,[GO.DLX.PRO.REP.12] Repurchase - Bundle Packag...,2021-12-21


In [5]:
df.columns

Index(['active_apr_2023', 'active_dec_2022', 'active_feb_2023',
       'active_jan_2023', 'active_mar_2023', 'active_nov_2022',
       'active_oct_2022', 'addon_1', 'addon_2', 'age', 'center', 'city',
       'contract_duration', 'core_product', 'dob', 'email', 'end_date',
       'fp_date', 'fp_month', 'fp_year', 'gender', 'id', 'income', 'index',
       'is_cpt', 'is_next_contract', 'job', 'membership_code',
       'membership_status', 'phone', 'product', 'start_date'],
      dtype='object')

# Repurchase Rate

In [6]:
df_pivot= (df
    # filter to relevant date range
    .loc[(df['fp_date'] >= start_date) & (df['fp_date'] <= end_date)]
    
    .groupby(['fp_year', 'core_product', 'fp_month'])
    .agg(
        fp_for_next_contract= ('is_next_contract', 'sum')
    )

    # filter only for core product
    .reset_index()
    .loc[lambda df_: df_['core_product'].isin(['Deluxe', 'VIP', 'Go'])]

    #pivot
    .pivot(index= ['fp_year', 'fp_month'], columns= ['core_product'])

    # add total column
    .assign(
        Total= lambda df_: df_.sum(axis= 1)
    )
)
df_pivot

fp_for_next_contract        Total
core_product                        Deluxe Go VIP      
fp_year fp_month                                       
2022    05 / May 2022                    5  6   5    16
        06 / Jun 2022                    6  7   4    17
        07 / Jul 2022                   13  3   1    17
        08 / Aug 2022                   13  9   4    26
        09 / Sep 2022                    8  4   3    15
        10 / Oct 2022                    7  5   2    14
        11 / Nov 2022                   10  7   5    22
        12 / Dec 2022                    6  9   7    22
2023    01 / Jan 2023                    5  6   4    15
        02 / Feb 2023                    5  2   1     8
        03 / Mar 2023                    4  7   0    11
        04 / Apr 2023                    6  3   4    13
        05 / May 2023                    1  1   0     2

In [11]:
# import os
# path= 'output/member_next_contract.xlsx'
# if not os.path.exists(path):
#     df_pivot.to_excel(path, engine='xlsxwriter', index= True)
#     print('File saved.')
# else:
#     print('File already exist.')

File already exist.
